In [9]:
import altair as alt
import pandas as pd
import altair_viewer
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

Measure = ['Quantity','Profit','Discount','Sales']

def filterDate(Dimension,typ): #Date inly

    df[Dimension] = pd.to_datetime(df[Dimension],format='%d/%m/%Y')

    if typ == 'year':
        s = str(Dimension+' year')
        df[s] = df[Dimension].dt.year
        return df[s]
    elif typ == 'month':
        s = str(Dimension+' month')
        df[s] = df[Dimension].dt.month
        return df[s]
    elif typ == 'day':
        s = str(Dimension+' day')
        df[s] = df[Dimension].dt.day
        return df[s]

def rangeScale(Di,Meas):
    fil = Meas[1]
    if fil == 'average':
        fil = 'mean'

    if len(Di) == 1:
        if (type(Di[0]) == type(['list'])):
            x = str(Di[0][0]+' '+Di[0][1])
        else:
            x = Di[0]
        #print([x])
        tmax = df.groupby([x], as_index=False)[Meas[0]].agg(fil).max()[1]
        tmin = df.groupby([x], as_index=False)[Meas[0]].agg(fil).min()[1]
        
        if tmin > 0:
            tmin = 0
        elif tmax < 0 :
            tmax = 0
        return [tmin,tmax]

    elif (type(Di[0]) == type(['list'])) and (type(Di[1]) == type(['list'])):
        x = str(Di[1][0]+' '+Di[1][1])
        col = str(Di[0][0]+' '+Di[0][1])
    elif type(Di[0]) == type(['list']):
        x = Di[1]
        col = str(Di[0][0]+' '+Di[0][1])
    elif type(Di[1]) == type(['list']):
        x = str(Di[1][0]+' '+Di[1][1])
        col = Di[0]
    else:
        x = Di[1]
        col = Di[0]
    #print(col,x,Meas[0])
    tmax = df.groupby([col,x], as_index=False)[Meas[0]].agg(fil).max()[2]
    tmin = df.groupby([col,x], as_index=False)[Meas[0]].agg(fil).min()[2]
    
    if tmin > 0:
        tmin = 0
    elif tmax < 0 :
        tmax = 0
    
    return [tmin,tmax]

def functionRC(row,column):
    lr = []
    lc = []

    for r in row:
        if type(r) == type(['list']):
            s = str(r[1]+'('+r[0]+')')
            lr.append(s)
        else:
            lr.append(r)

    for c in column:
        if type(c) == type(['list']):
            s = str(c[1]+'('+c[0]+')')
            lc.append(s)
        else:
            lc.append(c)
    
    return [lr,lc]

def plotBar(row,column,meas,di,mes): #edit and add dimen in 3 dimen
    #print(row,column,mes)
    print(di,meas)
    l = functionRC(row,column)
    lr = l[0]
    lc = l[1]
    if len(lr) == 2 and len(lc) == 0:           # 1 dimension and Measurement on row
        c = alt.Chart(df).mark_bar().encode(
            y= alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
            tooltip = [lr[-2],lr[-1]]
        ).facet(row=lr[-2]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
    
    elif len(lr) == 0 and len(lc) == 2:         # 1 dimension and Measurement on column
        c = alt.Chart(df).mark_bar().encode(
            x= alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
            tooltip = [lc[-2],lc[-1]]
        ).facet(column=lc[-2]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
    
    elif len(lr) == 1 and len(lc) == 1:         #1 dimension and Measurement with row and column
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=lr[-1]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:             
        if mes == 'row':
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lr[-2],lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lr[-2],lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 1 and len(lc) == 2:     #dimension2 date error
                 
        if mes == 'row':   
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 2 and len(lc) == 2:             ###################
        if mes == 'row':
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lc[-2],lr[-1],lc[-2],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lc[-2],lr[-1],lc[-2],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c


    elif len(lr) == 1 and len(lc) == 3:
        if mes == 'row':                
            c = alt.Chart(df).mark_bar().encode(        #all Dimen on Col or Row
                x=lc[-2],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                color = lc[-1],
                tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
            ).facet(column=lc[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            if column[2][0] in Measure:
                print('x=',lc[-2],'y=',lr[-1],'color=',lc[-1],'col=',lc[-3])
                c = alt.Chart(df).mark_bar().encode(
                    x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                    y=lr[-1],
                    color = lc[-2],
                    tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
                ).facet(column=lc[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c
                
            else:
                c = alt.Chart(df).mark_bar().encode(
                    x=alt.X(lc[-2],scale=alt.Scale(domain=rangeScale(di,meas))),
                    y=lr[-1],
                    color = lc[-1],
                    tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
                ).facet(column=lc[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c

    elif len(lr) == 3 and len(lc) == 1:
        if mes == 'row':
            if row[2][0] in Measure:
                c = alt.Chart(df).mark_bar().encode(
                    x=lc[-1],
                    y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                    color = lr[-2],
                    tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
                ).facet(row=lr[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c
            else:
                c = alt.Chart(df).mark_bar().encode(
                    x=lc[-1],
                    y=alt.Y(lr[-2],scale=alt.Scale(domain=rangeScale(di,meas))),
                    color = lr[-1],
                    tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
                ).facet(row=lr[-3]
                ).resolve_scale(x = 'independent',y = 'independent')
                return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-2],
                color = lr[-1],
                tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
            ).facet(row=lr[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 3 and len(lc) == 2:
        #print('x=',lc[-1],'y=',lr[-1],'color=',lr[-2],'row=',lr[-3],'column=',lc[-2])
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
            color = lr[-2],
            tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
        ).facet(row=lr[-3] , column = lc[-2]
        ).resolve_scale(y = 'independent')
        return c

    elif len(lr) == 2 and len(lc) == 3: 
        c = alt.Chart(df).mark_bar().encode(
            x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
            y=lr[-1],
            color = lc[-2],
            tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
        ).facet(column=lc[-3] , row = lr[-2]
        ).resolve_scale(x = 'independent')
        return c

    
        
def plotLine(row,column,meas,di,mes):

    print(di,meas)
    l = functionRC(row,column)
    lr = l[0]
    lc = l[1]

    if len(lr) == 1 and len(lc) == 1:         #1 dimension and Measurement with row and column
        c = alt.Chart(df).mark_line(point=True).encode(
            x=alt.X(lc[-1]),
            y=alt.Y(lr[-1]),
            tooltip = [lc[-1],lr[-1]]
        ).resolve_legend(size='independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:             
        if mes == 'row':
            c = alt.Chart(df).mark_line(point=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lc[-1],lr[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_line(point=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 1 and len(lc) == 2:     #dimension2 date error
        if mes == 'row':   
            c = alt.Chart(df).mark_line(point=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lc[-1],lr[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_line(point=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 2 and len(lc) == 2:             ###################
        if mes == 'row':
            c = alt.Chart(df).mark_line(point=True).encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lc[-1],lr[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_line(point=True).encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lr[-1],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

def plotPie(row,column,meas,di,mes):
    l = functionRC(row,column)
    lr = l[0]
    lc = l[1]
    
    '''base = alt.Chart(df).mark_arc().encode(
        theta=alt.Theta(str(fil+'('+Mes+'):Q')), 
        color=alt.Color(s, type="nominal"), 
        tooltip = [s,str(fil+'('+Mes+'):Q')]
    ).resolve_scale(theta = 'independent',color = 'independent')'''
    if len(lr) == 1 and len(lc) == 1:
        if mes == 'row':
            M = lr[-1]
            Di = lc
        else:
            M = lc[-1]
            Di = lr
        c = alt.Chart(df).mark_arc().encode(
            alt.Theta(M), 
            alt.Color(Di[-1], type="nominal"),
            tooltip = [M,Di[-1]]
        ).resolve_scale(theta = 'independent',color = 'independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:             
        if mes == 'row':
            c = alt.Chart(df).mark_arc().encode(
                alt.Theta(lr[-1]), 
                alt.Color(lc[-1], type="nominal"),
                alt.Row(lr[-2]),
                tooltip = [lr[-2],lc[-1],lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_arc().encode(
                alt.Theta(lc[-1]), 
                alt.Color(lr[-1], type="nominal"),
                alt.Row(lr[-2]),
                tooltip = [lr[-2],lc[-1],lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c

    elif len(lr) == 1 and len(lc) == 2:     #dimension2 date error
        if mes == 'row':   
            c = alt.Chart(df).mark_arc().encode(
                alt.Theta(lr[-1]), 
                alt.Color(lc[-1], type="nominal"),
                alt.Column(lc[-2])
                tooltip = [lc[-2],lc[-1],lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_arc().encode(
                alt.Theta(lc[-1]), 
                alt.Color(lr[-1], type="nominal"),
                alt.Column(lc[-2]),
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c

    elif len(lr) == 2 and len(lc) == 2:             ###################
        if mes == 'row':
            c = alt.Chart(df).mark_arc().encode(
                alt.Theta(lr[-1]), 
                alt.Color(lc[-1], type="nominal"),
                alt.Row(lr[-2]),
                alt.Column(lc[-2]),
                tooltip = [lr[-2],lc[-2],,lc[-1]lr[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_arc().encode(
                alt.Theta(lc[-1]), 
                alt.Color(lr[-1], type="nominal"),
                alt.Row(lr[-2]),
                alt.Column(lc[-2]),
                tooltip = [lr[-2],lc[-2],lr[-1],lc[-1]]
            ).resolve_scale(theta = 'independent',color = 'independent')
            return c

    return c

def plotGraph(row,column,tp):

    if tp == 'line':
        mes = 'col'
        for r in row:
            if type(r) == type(['list']):
                if r[0] in Measure:
                    mes = 'row'
        #print(mes)
        if mes == 'row':
            chart = []
            if type(row[0]) == type(['list']):  #Datetime and Meas
                if row[0][0] not in Measure:    #Datetime
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotLine([row[0],row[r+1]],column,row[r+1],l,mes))
                    return alt.vconcat(*chart)
                else:                           #Measure
                    for r in range(len(row)):
                        chart.append(plotLine([row[r]],column,row[r],[*column],mes))
                    return alt.vconcat(*chart)
            else:                               #Dimension
                l = [*column]
                l.append(row[0])
                for r in range(len(row)-1):
                    chart.append(plotLine([row[0],row[r+1]],column,row[r+1],l,mes))
                return alt.vconcat(*chart)
        else:
            chart = []
            if type(column[0]) == type(['list']):
                if column[0][0] not in Measure:
                    l = [*row]
                    l.append(column[0])
                    for c in range(len(column)-1):
                        chart.append(plotLine(row,[column[0],column[c+1]],column[c+1],l,mes))
                    return alt.vconcat(*chart)
                else:
                    for c in range(len(column)):
                        chart.append(plotLine(row,[column[c]],column[c],[*row],mes))
                    return alt.vconcat(*chart)
            else:
                l = [*row]
                l.append(column[0])
                for c in range(len(column)-1):
                    chart.append(plotLine(row,[column[0],column[c+1]],column[c+1],l,mes))
                return alt.vconcat(*chart)

    elif tp == 'bar':
        mes = 'col'
        for r in row:
            if type(r) == type(['list']):
                if r[0] in Measure:
                    mes = 'row'
        #print(mes)
        if mes == 'row':
            chart = []
            if type(row[0]) == type(['list']):  #Datetime and Meas
                if row[0][0] not in Measure:    #Datetime
                    if row[1][0] not in Measure:       #4Dimension (Can't plot)
                        print('4 Dimension')
                        l = [*column]
                        l.append(row[0])
                        l.append(row[1])
                        for r in range(len(row)-2):
                            chart.append(plotBar([row[0],row[1],row[r+2]],column,row[r+2],l,mes))
                        return alt.vconcat(*chart)
                    else:
                        l = [*column]
                        l.append(row[0])
                        for r in range(len(row)-1):
                            chart.append(plotBar([row[0],row[r+1]],column,row[r+1],l,mes))
                        return alt.vconcat(*chart)
                else:                           #Measure
                    for r in range(len(row)):
                        chart.append(plotBar([row[r]],column,row[r],[*column],mes))
                    return alt.vconcat(*chart)
            else:                               #Dimension
                '''if (type(row[1]) == type(['list'])) and row[1][0] in Measure:
                    print('t')
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotBar([row[0],row[r+1]],column,row[r+1],l,mes))
                    return alt.vconcat(*chart)'''
                if row[1][0] not in Measure:       #4Dimension
                    l = [*column]
                    l.append(row[0])
                    l.append(row[1])
                    for r in range(len(row)-2):
                        chart.append(plotBar([row[0],row[1],row[r+2]],column,row[r+2],l,mes))
                    return alt.vconcat(*chart)
                else:
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotBar([row[0],row[r+1]],column,row[r+1],l,mes))
                    return alt.vconcat(*chart)
        else:
            chart = []
            if type(column[0]) == type(['list']):
                if column[0][0] not in Measure:
                    if column[1][0] not in Measure:
                        l = [*row]
                        l.append(column[0])
                        l.append(column[1])
                        for c in range(len(column)-2):
                            chart.append(plotBar(row,[column[0],column[1],column[c+2]],column[c+2],l,mes))
                        return alt.vconcat(*chart)
                    else:
                        l = [*row]
                        l.append(column[0])
                        for c in range(len(column)-1):
                            chart.append(plotBar(row,[column[0],column[c+1]],column[c+1],l,mes))
                        return alt.vconcat(*chart)
                else:
                    for c in range(len(column)):
                        chart.append(plotBar(row,[column[c]],column[c],[*row],mes))
                    return alt.vconcat(*chart)
            else:
                '''if (type(column[1]) == type(['list'])) and column[1][0] in Measure:
                    l = [*row]
                    l.append(column[0])
                    for c in range(len(column)-1):
                        chart.append(plotBar(row,[column[0],column[c+1]],column[c+1],l,mes))
                    return alt.vconcat(*chart)'''
                if column[1][0] not in Measure:
                    l = [*row]
                    l.append(column[0])
                    l.append(column[1])
                    for c in range(len(column)-2):
                        chart.append(plotBar(row,[column[0],column[1],column[c+2]],column[c+2],l,mes))
                    return alt.vconcat(*chart)
                else:
                    l = [*row]
                    l.append(column[0])
                    for c in range(len(column)-1):
                        chart.append(plotBar(row,[column[0],column[c+1]],column[c+1],l,mes))
                    return alt.vconcat(*chart)

    elif tp == 'pie':
        mes = 'col'
        for r in row:
            if type(r) == type(['list']):
                if r[0] in Measure:
                    mes = 'row'
        #print(mes)
        if mes == 'row':
            chart = []
            if type(row[0]) == type(['list']):  #Datetime and Meas
                if row[0][0] not in Measure:    #Datetime
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotPie([row[0],row[r+1]],column,row[r+1],l,mes))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
                else:                           #Measure
                    for r in range(len(row)):
                        chart.append(plotPie([row[r]],column,row[r],[*column],mes))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
            else:                               #Dimension
                l = [*column]
                l.append(row[0])
                for r in range(len(row)-1):
                    chart.append(plotPie([row[0],row[r+1]],column,row[r+1],l,mes))
                return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
        else:
            chart = []
            if type(column[0]) == type(['list']):
                if column[0][0] not in Measure:
                    l = [*row]
                    l.append(column[0])
                    for c in range(len(column)-1):
                        chart.append(plotPie(row,[column[0],column[c+1]],column[c+1],l,mes))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
                else:
                    for c in range(len(column)):
                        chart.append(plotPie(row,[column[c]],column[c],[*row],mes))
                    return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')
            else:
                l = [*row]
                l.append(column[0])
                for c in range(len(column)-1):
                    chart.append(plotPie(row,[column[0],column[c+1]],column[c+1],l,mes))
                return alt.vconcat(*chart).resolve_scale(theta = 'independent',color = 'independent')

df = pd.read_csv('Superstore.csv', encoding='windows-1252')

for d in ['Order Date','Ship Date']:
    filterDate(d,'year')
    filterDate(d,'month')
    filterDate(d,'day')

D1 = ['Order Date','quarter']
D2 = ['Ship Date','month']

D3 = 'Category'
D4 = 'Sub-Category'

D5 = 'Country/Region'
D6 = 'Region'
D7 = 'State'
D8 = 'City'

D9 = 'Ship Mode'
D10 = 'Segment'

M1 = ['Quantity','average']
M2 = ['Discount','sum']
M3 = ['Profit','sum']
M4 = ['Sales','sum']

column = [D10]
row = [M2,M3]

#column , row = row ,column

#plotBar(row,column,f1)
#plotLine(row,column,f1)
#plotPie(row,column,f1)
plotGraph(row,column,'pie')
#print(df.columns)

sum(Discount) ['Segment']
sum(Profit) ['Segment']


alt.VConcatChart(...)

In [ ]:
Measure = ['Quantity','Profit','Discount','Sales']
row = [M2,D1,M3,D5,D7]
column = [D6,D3,M1]
r = 0
c = 0
for i in row:
    if type(i) == type(['list']):
        if i[0] not in Measure:
            r += 1
    elif i not in Measure:
        r +=1

for j in column:
    if type(j) == type(['list']):
        if j[0] not in Measure:
            c += 1
    elif j not in Measure:
        c +=1
print(r,c)

In [ ]:
c = alt.Chart(df).mark_bar().encode(
    y = 'Segment',
    x= 'sum(Profit)',
    color = 'State'
).facet(row='Category'
).resolve_scale(x = 'independent',y = 'independent')
c


In [13]:
for d in ['Order Date','Ship Date']:
    x = filterDate(d,'year')
list(x.drop_duplicates())

[2019, 2018, 2017, 2020, 2021]